In [1]:
# https://stackoverflow.com/questions/21971449/how-do-i-increase-the-cell-width-of-the-jupyter-ipython-notebook-in-my-browser

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, train_test_split, LeaveOneOut
from sklearn.metrics import roc_curve, auc, plot_confusion_matrix, plot_precision_recall_curve, classification_report

from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

from tqdm.notebook import tqdm

plt.style.use('seaborn-whitegrid')

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
import os
import gc
import time
import copy
import torch
import model_utils as u
import model_evaluation as me

In [4]:
torch.use_deterministic_algorithms(True)
os.environ["CUBLAS_WORKSPACE_CONFIG"]=":16:8"

In [5]:
import warnings
warnings.simplefilter('ignore')

In [6]:
# https://pytorch.org/docs/stable/notes/randomness.html
seed = 322
u.set_all_seeds(seed)

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

---
---
---

In [8]:
PATH = "D:/CANCER BIOLOGY/DATASET/TCGA/FROM Xena/"

In [9]:
# https://stackoverflow.com/questions/18885175/read-a-zipped-file-as-a-pandas-dataframe
# https://www.analyticsvidhya.com/blog/2021/04/delimiters-in-pandas-read_csv-function/

df_luad = pd.read_csv(PATH+"TCGA.LUAD.sampleMap_HiSeqV2_PANCAN.gz", compression = "gzip", sep = "\t")
df_lusu = pd.read_csv(PATH+"TCGA.LUSC.sampleMap_HiSeqV2_PANCAN.gz", compression = "gzip", sep = "\t")

In [10]:
df, labels, columns = u.dataset_preprocess(df_luad, df_lusu)

,label,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,REM1,MTVR2,...,TULP2,NPY5R,GNGT2,GNGT1,TULP3,PTRF,BCL6B,GSTK1,SELP,SELS
TCGA-69-7978-01,1,0.125808,-1.29493,-0.112935,-1.41187,0.203922,0.0993901,-0.222094,0.504354,-0.423399,...,1.13472,-0.845117,1.76017,-1.28139,0.224623,0.863614,0.802173,0.108205,0.595367,-0.222712
TCGA-62-8399-01,1,0.561708,6.06917,-0.531035,-0.228672,0.0521219,-1.20601,-0.338894,1.44985,0.0394006,...,-0.286078,-0.055517,-0.0282335,0.0480102,0.295223,0.317114,1.07907,-0.782695,3.11427,-0.388912
TCGA-78-7539-01,1,-0.237592,3.58147,0.592065,-0.108372,-0.499978,-0.0254099,0.163006,0.131654,-0.0508994,...,2.20992,-1.58712,1.56537,2.63871,0.0491232,-1.24309,-1.28323,0.0341054,0.571467,0.233588
TCGA-50-5931-11,1,-1.18049,3.92767,0.291065,-0.043472,0.710822,1.05089,-0.564394,1.68395,0.398701,...,-0.748878,0.900483,1.90357,-1.28139,-0.549277,2.80271,2.25047,-0.540795,3.98597,0.370988
TCGA-73-4658-01,1,-0.656192,-0.525926,-0.531035,-0.156672,0.373522,0.44729,-0.438994,1.83155,-0.423399,...,-0.748878,-0.931417,1.91517,-0.17659,0.0423232,1.11071,1.51397,-0.454095,2.89317,0.193788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-85-A512-01,0,-1.28699,-1.86343,-0.531035,0.020328,-0.560978,0.51729,0.519806,-0.748146,-0.423399,...,-0.748878,-1.58712,-0.236033,1.41071,1.65372,0.434414,-0.491427,0.318305,-1.01143,0.392288
TCGA-85-8354-01,0,-1.84779,-1.29733,-0.531035,-0.280072,-0.560278,0.85009,-0.166294,-1.51925,0.602101,...,0.276622,-1.58712,-0.784133,1.46201,0.765023,1.24691,-0.0700266,-0.620695,-1.39033,0.918888
TCGA-O2-A5IB-01,0,0.568508,0.0259737,0.084865,-0.00937199,-0.136278,-1.41811,0.173506,-2.19405,0.622701,...,-0.408378,-1.24662,-1.94053,-1.28139,1.28172,-2.42559,-0.734427,-1.92679,-2.12163,-0.637412
TCGA-77-7335-01,0,-0.849892,-2.41413,0.192865,0.197228,0.443922,0.54789,0.0844055,1.01505,-0.423399,...,-0.748878,-1.58712,2.16037,1.19391,0.197023,1.55011,0.294773,0.369005,1.90927,0.117688


---
---
---

In [11]:
second_iter_path = 'D:/CANCER BIOLOGY/DATASET/TCGA/FROM Xena/SECOND_ITERATION/cumulative_lists_results'

In [12]:
cumulative_lists = os.listdir(second_iter_path+"/freq_geq_5")

In [13]:
def get_sklearn_classifiers():
    
    epochs = 100
    learning_rate = 1e-5
    batch_size_df = 32
    
    sklearn_classifiers = [

        MLPClassifier(hidden_layer_sizes=(512, 256, 128), random_state=seed, batch_size=batch_size_df, verbose=False, learning_rate_init=learning_rate, max_iter=epochs, alpha=0.02),  ## predict_proba
        LogisticRegression(random_state=seed, max_iter=epochs, penalty='elasticnet', l1_ratio=0.04, solver='saga'),  ## predict_proba
        XGBClassifier(eta=0.1, max_depth=10, alpha=0.5, random_state=seed, verbosity=0),
        SVC(kernel='rbf', random_state=seed), ## decision_func
    ]
    return sklearn_classifiers
    

def compute_k_fold(X_train, y_train):
    k = 10
    kfold = StratifiedKFold(n_splits=10, shuffle=False)
    sklearn_classifiers = get_sklearn_classifiers()
    
    train_acc_dict={
        'MLPClassifier':0.,
        'LogisticRegression':0.,
        'XGBClassifier': 0.,
        'SVC':0.
    }
    test_acc_dict={
        'MLPClassifier':0.,
        'LogisticRegression':0.,
        'XGBClassifier': 0.,
        'SVC':0.
    }
    
    classifier_name = ['MLPClassifier','LogisticRegression', 'XGBClassifier', 'Support Vector Classifier']
    for fold, (train_index, test_index) in enumerate(kfold.split(X_train, y_train)):

        xtrain, xtest = X_train.iloc[X_train.index[train_index]], X_train.iloc[X_train.index[test_index]]
        ytrain, ytest = np.array(y_train)[train_index], np.array(y_train)[test_index]

        scaler = StandardScaler()
        xtrain_scaled = scaler.fit_transform(xtrain)
        xtest_scaled = scaler.transform(xtest)
        
        heading = HTML("<b>Fold: {}</b>".format(fold+1))
        display(heading)

        for classifier in tqdm(sklearn_classifiers):
            classifier.fit(xtrain_scaled, ytrain)
            train_acc_dict[f'{type(classifier).__name__}'] += classifier.score(xtrain_scaled, ytrain)
            test_acc_dict[f'{type(classifier).__name__}'] += classifier.score(xtest_scaled, ytest)
    
    temp_pd = pd.DataFrame({'Avg Train Accuracy':list(train_acc_dict.values()), 'Avg Test Accuracy':list(test_acc_dict.values())}, index=classifier_name)
    temp_pd = temp_pd/k
    display(temp_pd)
    ruler = HTML("<hr><hr>")
    display(ruler)
    return temp_pd


In [14]:
df.reset_index(drop=True, inplace=True)
for ls in cumulative_lists:
    gene_list=[]
    with open(second_iter_path+'/freq_geq_5/'+ls, "r") as file:
        for gene in file:
            gene=gene.strip()
            gene_list.append(gene)
            

    X = df[df.columns.intersection(gene_list)]
    y = labels
    ruler = HTML("<b>LIST: {}".format(ls))
    display(ruler)
    temp_pd = compute_k_fold(X, y)
    temp_pd.to_csv(second_iter_path+'/accuracies/'+ls+'.csv')
            


  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

,Avg Train Accuracy,Avg Test Accuracy
MLPClassifier,0.951580,0.940653
LogisticRegression,0.944395,0.932672
XGBClassifier,0.999311,0.938859
Support Vector Classifier,0.950989,0.940645


---
---
---